In [173]:
#%%
#Assignment 2

import os
import numpy as np
import pandas as pd
import spacy
import nltk
import csv
import torch
from collections import defaultdict, Counter
import random
import sklearn.metrics as sk
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from random import seed
from random import randrange


#Task 1: Class distribution
train_file = 'olid-train.csv'
df = pd.read_csv(train_file) 
count_lab0 = 0
count_lab1 = 0
for item in df:
    for i in df.labels:
        if i==0:
            count_lab0 += 1
        if i==1:
            count_lab1 += 1
            #print()
tot_labels=len(df.labels)
print('Number of labels with 0:', count_lab0, 'and', (count_lab0*100/tot_labels), '%')   # 0 = NOT (non-offensive messages)
print('Number of labels with 1:', count_lab1, 'and', (count_lab1*100/tot_labels), '%')   # 1 = OFF (offensive messages)
print('Twitter example of label 0: \n', df.loc[df['labels']==0, 'text'].iloc[0])
print('Twitter example of label 1: \n', df.loc[df['labels']==1, 'text'].iloc[0])






##############################################################################
#Task 2: Baselines
#Definitions:
#precision = True Positives / (True Positives + False Positives)
#recall = True Positives / (True Positives + False Negatives)
#F1 = 2 * (Precision * Recall) / (Precision + Recall)
#macro-average = does not take class imbalance into account
#weighted average = average over classes, weighted by class size (support)

test_file='olid-test.csv'
df2 = pd.read_csv(test_file)
labels = df["labels"]
class_count = labels.value_counts()
class_freq = class_count / sum(class_count)




#2.1: Random baseline that randomly assigns one of the 2 classification labels
prf1_list = ['precision', 'recall', 'f1-score']
random_base = {0: {'precision': [], 'recall': [], 'f1-score': []},
                  1: {'precision': [], 'recall': [], 'f1-score': []}}
random.seed(2)   #seed is randomly chosen 
random_labels = np.random.randint(0, 2, len(labels))
temp_result = sk.classification_report(labels, random_labels, output_dict=True)
for i in range(2):
    for m in prf1_list:
        random_base[i][m].append(temp_result[str(i)][m])

#2.2: Majority baseline that always assigns the majority class
majority_base = {0: {'precision': [], 'recall': [], 'f1-score': []},
                    1: {'precision': [], 'recall': [], 'f1-score': []}}
major_labels = np.full_like(labels, np.argmax(class_count))
temp_result = sk.classification_report(labels, major_labels, output_dict=True, zero_division=0)
for i in range(2):
    for m in prf1_list:
        majority_base[i][m].append(temp_result[str(i)][m])

        
        
        
# Create dictionary with results
for m in prf1_list:
    results_random = []
    results_major = []
    for i in range(2):
        random_base[i][m] = round(np.mean(random_base[i][m]), 2)
        results_random.append(random_base[i][m])
        majority_base[i][m] = round(np.mean(majority_base[i][m]), 2)
        results_major.append(majority_base[i][m])
    random_base['macro_' + m] = round(np.mean(results_random), 2)
    random_base['weighted_' + m] = round(np.average(results_random, weights=class_freq.values), 2)
    majority_base['macro_' + m] = round(np.mean(results_major), 2)
    majority_base['weighted_' + m] = round(np.average(results_major, weights=class_freq.values), 2)

print('Random baseline:', random_base, '\n')
print('Majority baseline:', majority_base)



Number of labels with 0: 26520 and 200.3021148036254 %
Number of labels with 1: 13200 and 99.69788519637463 %
Twitter example of label 0: 
 Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT
Twitter example of label 1: 
 @USER She should ask a few native Americans what their take on this is.
Random baseline: {0: {'precision': 0.67, 'recall': 0.5, 'f1-score': 0.58}, 1: {'precision': 0.34, 'recall': 0.51, 'f1-score': 0.41}, 'macro_precision': 0.5, 'weighted_precision': 0.56, 'macro_recall': 0.5, 'weighted_recall': 0.5, 'macro_f1-score': 0.5, 'weighted_f1-score': 0.52} 

Majority baseline: {0: {'precision': 0.67, 'recall': 1.0, 'f1-score': 0.8}, 1: {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0}, 'macro_precision': 0.34, 'weighted_precision': 0.45, 'macro_recall': 0.5, 'weighted_recall': 0.67, 'macro_f1-score': 0.4, 'weighted_f1-score': 0.53}


In [109]:
# Model

nlp = spacy.load('en_core_web_sm')

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-cased',num_labels = 5)
model.cuda()

In [174]:
# Part B: Error analysis with checklist

#New train set - subset 100 instances
train_set='olid-subset-diagnostic-tests.csv'
subset = pd.read_csv(train_set)
labels = df["labels"]
class_count = labels.value_counts()
class_freq = class_count / sum(class_count)

subset

,id,text,labels
0,89200,@USER @USER Who the hell does he think he is?,1
1,71294,#BREAKING. #Greece: Molotov cocktails fly afte...,1
2,55633,"#OrrinHatch I can’t believe this sexist , clue...",1
3,16856,@USER @USER I'll use that one the next time im...,1
4,26072,0-1 lost my acca on the first fucking fight cba,1
...,...,...,...
95,45518,@USER He is obviously getting suspended. He is...,0
96,51610,#Canada - EXCLUSIVE: #Trudeau #Liberals leave ...,0
97,26758,@USER @USER ...than why did you show us how ho...,0
98,30718,@USER @USER @USER You have yet to answer what ...,0


In [204]:
#Perturbations

import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb

In [205]:
# jupyter nbextension install --py --sys-prefix checklist.viewer
# jupyter nbextension enable --py --sys-prefix checklist.viewer

In [206]:
#Q5.



In [207]:
# Load SA  

import checklist
from checklist.test_suite import TestSuite
# suite_path = 'release_data/sentiment/sentiment_suite.pkl'
#suite = TestSuite.from_file(suite_path)

In [208]:
pred_path = 'release_data/sentiment/predictions/bert'
#suite.run_from_file(pred_path, overwrite=True)
#suite.summary() # or suite.visual_summary_table()


In [209]:
# Perturbations
import spacy
nlp = spacy.load('en_core_web_sm')
pdataset = list(nlp.pipe(subset))
ret = Perturb.perturb(pdataset, Perturb.change_names, n=2)
ret.data

[]

In [212]:
# Q6. Adding negations

#neg = Perturb.add_negation(subset['text'])
#neg.subset

negated_text = Perturb.perturb(pdataset, Perturb.add_negation, keep_original=False)
negated_text

MunchWithAdd({'data': []})

In [202]:
# subset_text=subset['text']
# neg_list=[]
# for i in range(0,len(subset)):
  #  neg_list.append(Perturb.add_negation(all_text1[i]))
    
#subset_text=list(nlp.pipe)

In [ ]:
# Q7. Negated examples

In [134]:
#From lexicon -> ‘I hate {nationality}’, ‘I don’t hate {nationality}’, ‘I hate {religion}’, ‘I don’t hate {religion}’ 

editor = Editor(language='English')
ret = editor.template('I hate {mask}', remove_duplicates=True)
list(np.random.choice(ret.data, 10))

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

['I hate work',
 'I hate her',
 'I hate politicians',
 'I hate him',
 'I hate school',
 'I hate you',
 'I hate inequality',
 'I hate change',
 'I hate hate',
 'I hate capitalism']

In [135]:
ret = editor.template('I don\'t hate {mask}', remove_duplicates=True)
list(np.random.choice(ret.data, 10))

["I don't hate Russians",
 "I don't hate Jews",
 "I don't hate bullies",
 "I don't hate God",
 "I don't hate abortion",
 "I don't hate Trump",
 "I don't hate work",
 "I don't hate Americans",
 "I don't hate racism",
 "I don't hate you"]

In [136]:
ret = editor.template('I hate {nationality}', remove_duplicates=True)
list(np.random.choice(ret.data, 10))

['I hate Turks and Caicos Island',
 'I hate Rwandan',
 'I hate Jamaican',
 'I hate Kenyan',
 'I hate Saint Lucian',
 'I hate British',
 'I hate Serbian',
 'I hate Bissau-Guinean',
 'I hate Ivorian',
 'I hate Gambian']

In [137]:
ret = editor.template('I don\'t hate {nationality}', remove_duplicates=True)
list(np.random.choice(ret.data, 10))

["I don't hate Djiboutian",
 "I don't hate Fijian",
 "I don't hate Basotho",
 "I don't hate Cameroonian",
 "I don't hate Somali",
 "I don't hate Togolese",
 "I don't hate Azerbaijani",
 "I don't hate Gabonese",
 "I don't hate Qatari",
 "I don't hate Bangladeshi"]

In [145]:
ret = editor.template('I hate {religion}', remove_duplicates=True)
list(np.random.choice(ret.data, 10))

['I hate Judaism',
 'I hate Agnosticism',
 'I hate Buddhism',
 'I hate Christianity',
 "I hate Baha'i",
 'I hate Agnosticism',
 'I hate Agnosticism',
 'I hate Sikhism',
 'I hate Zoroastrianism',
 'I hate Hinduism']

In [148]:
ret = editor.template('I don\'t hate {religion}', remove_duplicates=True)
list(np.random.choice(ret.data, 10))

["I don't hate Atheism",
 "I don't hate Islam",
 "I don't hate Christianity",
 "I don't hate Agnosticism",
 "I don't hate Confucianism",
 "I don't hate Shintoism",
 "I don't hate Atheism",
 "I don't hate Judaism",
 "I don't hate Shintoism",
 "I don't hate Taoism"]